In [2]:
import numpy as np
from random import shuffle

#sigmoid function
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

#Main class that represents the network
class Network(object):
    def __init__(self, sizes): #sizes is a list that contains the number of neurons in each respective layer
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y,1) for y in sizes[1:]] #randomly initializing the biases afther the firs layer (because the first one is the input layer)
        self.weights = [np.random.rand(y, x) for x, y in zip(sizes[:-1], sizes[1:])] #randomly initializing the weights
        
    def feedForward(self, a):
        "Return the output of the network if 'a' is input."
        for b, w in zip(self.biases, self.weights): #zip aggregates iterables into tuples
            a = sigmoid(np.dot(w,a)+b) #np.dot = produto vetorial
            
    def SGD(self, trainingData, epochs, miniBatchSize, eta, testData = None): #Stochastic gradient descent
        #eta is the learning rate n
        #"Train the neural network using mini-batch stochastic gradient descent. the trainingData is a list of tuples representing the training inputs and the desired outputs."
        #"If testData is provided then the network will be evaluated against the test data after each epoch, and partial progress printed out. Thi is useful for tracking progress but it slows things down substantialy"
        if testData: nTest = len(testData)
        n = len(trainingData)
        for j in range(epochs):
            random.shuffle(trainingData) #shuffle the order of trainingData
            #now that the trainingData has been shuffled we will partition it in miniBatches
            miniBatches = [trainingData[k:k+miniBatchSize] for k in range(0, n, miniBatchSize)] #This range goes from 0 to n in miniBatchSize jumps
            for miniBatch in miniBatches:
                self.updateMiniBatch(miniBatch, eta) #calls a function that updates the weights and biases of the network according to a single iteration of gradient descent useing just the data in the miniBatch
            if testData:
                print("Epoch {0}: {1} / {2}".format(j, self.evaluate(testData), nTest))
            else:
                print("Epoch {0} complete".format(j))
                
    def updateMiniBatch(self, miniBatch, eta):
        #"Update the network's weights and biases applying gradient descent using backpropagation to a single mini batch"
        #The 'miniBatch' is a list of tuples and 'eta' is the learning rate
        nabla_b = [np.zeros(b.shape) for b in self.biases] #gradient of biases
        nabla_w = [np.zeros(w.shape) for w in self.weights] #gradient of weights
        for x,y in miniBatch: #computes the gradient for every training example on miniBatch
            delta_nabla_b, delta_nabla_w = self.backdrop(x,y) #invokes the backpropagation algorithm (!!!)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)] #updates the direction of the gradient vector
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(miniBatch))*nw for w,nw in zip(self.weights, nabla_w)] #updates the weights
        self.biases = [b-(eta/len(miniBatch))*nb for b, nb in zip(self.biases, nabla_b)]

#### The code for the "backdrop" function is still missing